<div  style="display: table; width: 100%; color: #034ea2; background:#ffffff ; line-height: 100px;  text-align: center;" dir="rtl">
        <h2 style="display:table-cell;">جامعة القلمون  </h2>
        <h1 style="display:table-cell;"> <b>👨‍💻 اويس الحمود 👨‍💻</b></h1>
        <h2 style="display:table-cell;">201811330</h2>
    </div>

In [1]:
import pandas as pd
import re
import snowballstemmer 
import arabicstopwords.arabicstopwords as stp
import os


In [2]:
def remove_stopWords(sentence):
    terms=[]
    stopWords= set(stp.stopwords_list())
    for term in sentence.split() : 
        if term not in stopWords :
            terms.append(term)
    return " ".join(terms)

In [3]:
def normalize(text):
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    return(text)

In [4]:
ar_stemmer = snowballstemmer.stemmer('arabic')
def stem(sentence):
    return " ".join([ar_stemmer.stemWord(i) for i in sentence.split()])

In [5]:
def preProcessing(text):
    return stem(normalize(remove_stopWords(text)))

In [6]:
i=0
docs_df = pd.DataFrame(columns=["docno", "raw_text"])
for filename in os.listdir("..\\mydata"):
    i+=1 
    filepath = os.path.join("..\\mydata", filename) 
    with open(filepath, "r") as file: 
        data = file.read()
    docs_df.loc[len(docs_df)]=[f'd{i}',data]
         
pd.set_option('display.max_colwidth',150)
docs_df.head()

,docno,raw_text
0,d1,هذه هي المحاضرة الثانية من مقرر نظم استرجاع المعلومات
1,d2,المحاضرة باللغة العربية لطلاب جامعة القلمون
2,d3,كان الجو غائماً جزئياً
3,d4,نأمل أن تفيد هذا المقرر طلاب جامعة القلومن
4,d5,هل أنتم سعداء بهذه التجربة


In [7]:
docs_df['text']=docs_df['raw_text'].apply(preProcessing)
docs_df.head()

,docno,raw_text,text
0,d1,هذه هي المحاضرة الثانية من مقرر نظم استرجاع المعلومات,محاضره ثانيه مقرر نظم استرجاع معلوم
1,d2,المحاضرة باللغة العربية لطلاب جامعة القلمون,محاضره لغه عربيه لطلاب جامع قلم
2,d3,كان الجو غائماً جزئياً,الجو غاءم جزءي
3,d4,نأمل أن تفيد هذا المقرر طلاب جامعة القلومن,نامل تفيد مقرر طلاب جامع قلوم
4,d5,هل أنتم سعداء بهذه التجربة,سعداء تجربه


In [16]:
terms=[]
for doc in docs_df['text']:
    for term in doc.split():
        terms.append(term)
terms

['محاضره',
 'ثانيه',
 'مقرر',
 'نظم',
 'استرجاع',
 'معلوم',
 'محاضره',
 'لغه',
 'عربيه',
 'لطلاب',
 'جامع',
 'قلم',
 'الجو',
 'غاءم',
 'جزءي',
 'نامل',
 'تفيد',
 'مقرر',
 'طلاب',
 'جامع',
 'قلوم',
 'سعداء',
 'تجربه']

In [17]:
unique_terms=[]
for d in terms :
    if d not in unique_terms:
        unique_terms.append(d)
unique_terms

['محاضره',
 'ثانيه',
 'مقرر',
 'نظم',
 'استرجاع',
 'معلوم',
 'لغه',
 'عربيه',
 'لطلاب',
 'جامع',
 'قلم',
 'الجو',
 'غاءم',
 'جزءي',
 'نامل',
 'تفيد',
 'طلاب',
 'قلوم',
 'سعداء',
 'تجربه']

In [30]:
index={}
for i, doc in enumerate(docs_df['text']):
    for t in terms:
        if t not in index:
            index[t] = []
        if(t in doc):
            index[t].append(docs_df.loc[i,'docno'])
index

{'محاضره': ['d1', 'd1', 'd2', 'd2'],
 'ثانيه': ['d1'],
 'مقرر': ['d1', 'd1', 'd4', 'd4'],
 'نظم': ['d1'],
 'استرجاع': ['d1'],
 'معلوم': ['d1'],
 'لغه': ['d2'],
 'عربيه': ['d2'],
 'لطلاب': ['d2'],
 'جامع': ['d2', 'd2', 'd4', 'd4'],
 'قلم': ['d2'],
 'الجو': ['d3'],
 'غاءم': ['d3'],
 'جزءي': ['d3'],
 'نامل': ['d4'],
 'تفيد': ['d4'],
 'طلاب': ['d2', 'd4'],
 'قلوم': ['d4'],
 'سعداء': ['d5'],
 'تجربه': ['d5']}

In [31]:
boolean_operators = {'AND', 'OR'}
def boolean_operator_processing(bool_operator,prevV,nextV):
    if bool_operator == "AND":
        return set(prevV)&set(nextV)
    elif bool_operator=="OR" :
        return set(prevV)|set(nextV)

In [34]:
q='المحاضرة OR جامعة'
q=preProcessing(q)
q=q.split()
v=index[q[0]]
u=index[q[2]]
boolean_operator_processing(q[1],v,u)


{'d1', 'd2', 'd4'}